# OpenAI 매개변수

# 개요
OpenAI 모델에 요청을 보낼 때 여러 매개변수를 사용하여 모델의 동작과 출력을 제어할 수 있습니다. \
이러한 매개변수를 이해하면 텍스트 생성, 질문 응답 또는 기타 사용 사례에 맞게 응답을 세부 조정할 수 있습니다.

더 자세한 예제는 공식 문서를 참조하세요: [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-12-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
SEED = 123

# 매개변수: max_tokens
**설명**: 생성할 응답의 최대 토큰 수를 설정합니다. \
**기본값**: 16 \
**예제**: max_tokens=50

프롬프트의 토큰 수와 max_tokens의 합은 모델의 컨텍스트 길이를 초과할 수 없습니다. \
gpt-4o-mini의 경우 16,384 tokens이며 모델에 따라 다릅니다. 

In [2]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "최고의 반려동물은 "}],
                    max_tokens=max_tokens
    )
    return response.choices[0].message.content

tokens = [32, 64, 120, 200]
for token in tokens:
    print(f"Max Tokens: {token}\n")
    print(call_openai_with_max_tokens(token))
    print("\n" + "-"*80 + "\n")

Max Tokens: 32

최고의 반려동물은 사람마다 다를 수 있지만, 일반적으로 개인의 생활 방식, 성격, 취향에 따라 달라집

--------------------------------------------------------------------------------

Max Tokens: 64

"최고의 반려동물"은 개인의 취향과 생활 방식에 따라 다를 수 있습니다. 어떤 사람에게는 강아지가 최고의 반려동물일 수 있고, 다른 사람에게는 고양이나 햄스터, 또는 물고기가 더 잘 맞을 수 있습니다. 중요한 것은

--------------------------------------------------------------------------------

Max Tokens: 120

최고의 반려동물은 사람마다 다를 수 있지만, 보통 개인의 성격, 생활 환경, 취향에 따라 달라집니다. 예를 들어, 활동적이고 외향적인 사람에게는 강아지가 좋은 반려동물이 될 수 있고, 조용하고 독립적인 성격이라면 고양이가 더 잘 맞을 수 있습니다. 또한, 관리가 비교적 쉬운 작은 반려동물을 원한다면 햄스터나 물고기, 토끼도 좋은 선택이 될 수 있습니다. 본인의 생활 패턴과 성

--------------------------------------------------------------------------------

Max Tokens: 200

최고의 반려동물은 사람마다 다를 수 있어요. 어떤 사람은 강아지를 최고의 반려동물로 생각하고, 어떤 사람은 고양이, 새, 토끼, 물고기 등을 좋아하기도 해요. 반려동물을 선택할 때는 자신의 생활 방식, 공간, 시간, 성격 등을 고려하는 것이 중요해요. 예를 들어, 활동적이고 외출이 잦은 사람은 적절히 산책이 필요한 강아지가 잘 맞을 수 있고, 혼자 있는 시간이 많은 사람은 독립적인 고양이가 더 어울릴 수 있죠. 가장 중요한 것은 반려동물과 주인이 서로 잘 맞고 행복하게 지내는 것입니다. 어떤 반려동물이든 사랑과 관심을 주면 

# 매개변수: temperature

**설명**: 출력의 무작위성을 제어합니다. 낮은 값은 출력을 더 결정론적으로 만들고, 높은 값은 무작위성을 증가시킵니다. \
**값 범위**: 0에서 1 \
**기본값**: 1 \
**예제**: temperature=0.7

높은 값은 모델이 더 창의적인 출력을 생성하도록 합니다. \
창의적인 응용 프로그램에는 0.9를, 명확한 답변이 필요한 경우에는 0(최대 확률 샘플링)을 시도하세요.

---
**참고**: 일반적으로 이 매개변수 또는 top_p를 조정하되 둘 다 동시에 조정하지 않는 것을 권장합니다.


In [3]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    temperature = temperature
            )
        print(response.choices[0].message.content)

In [4]:
# Without seed and temperature, the response is different each time
call_openai(10, '최고의 반려동물은 ')

최고의 반려동물은 사람마다 다르지만, 자신과 잘 맞고 함께 행복할 수 있는 동물이 최고의 반려동물이라고 할 수 있습니다. 예를 들어, 활동적인 사람에게는 에너지 넘치는 강아지가, 조용하고 독립적인 생활
최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 최고의 반려동물로 생각할 수 있고, 어떤 사람은 고양이나 새, 토끼, 물고기 등 다른 동물을 더 좋아할 수도 있습니다. 반려동물을
최고의 반려동물은 사람마다 다를 수 있습니다. 개인의 성격, 생활 방식, 취향에 따라 적합한 반려동물이 다르기 때문입니다. 예를 들어, 활동적이고 산책을 자주 할 수 있는 사람에게는 강아
최고의 반려동물은 사람마다 다를 수 있습니다. 개, 고양이, 토끼, 햄스터, 새 등 각 동물마다 특성과 장점이 다르기 때문입니다. 자신과 생활 패턴, 성격, 환경에 잘 맞는 반
최고의 반려동물은 사람마다 다를 수 있지만, 보통 강아지나 고양이가 많이 사랑받습니다. 강아지는 충성심이 강하고 함께 활동하기 좋아하며, 고양이는 독립적이면서도 애교가 많아 집에서
최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 최고의 반려동물로 생각하고, 어떤 사람은 고양이, 새, 토끼, 햄스터 등 다른 동물을 더 좋아할 수 있습니다. 중요한 것은 자신의 생활 스타일
최고의 반려동물은 사람마다 다를 수 있습니다. 개인의 라이프스타일, 성격, 환경에 따라 적합한 반려동물이 다르기 때문입니다. 예를 들어, 활동적이고 산책을 좋아하는 사람에게는 강아지가 잘
최고의 반려동물은 사람마다 다를 수 있지만, 보통 강아지, 고양이, 햄스터, 토끼 등이 많이 사랑받습니다. 각 반려동물은 성격, 관리 방법, 생활 환경에 따라 장단점이 있으니 자신의
최고의 반려동물은 사람마다 다를 수 있어요. 개인의 성격, 생활 환경, 취향에 따라 고양이, 강아지, 토끼, 햄스터, 새 등 다양한 동물이 최고의 반려동물이 될 수 있답니다. 예
최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 최고의 반려동물

In [5]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, '최고의 반려동물은 ', temperature = 0, use_seed=True)

최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 좋아하고, 어떤 사람은 고양이나 토끼, 새, 물고기 등을 더 좋아할 수 있지요. 반려동물을 선택할 때는 자신의 생활 스타일, 공간
최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 좋아하고, 어떤 사람은 고양이나 토끼, 새, 물고기 등을 더 좋아할 수 있지요. 반려동물을 선택할 때는 자신의 생활 스타일, 공간
최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 좋아하고, 어떤 사람은 고양이를 더 좋아하기도 하죠. 반려동물을 선택할 때는 자신의 생활 스타일, 성격, 환경 등을 고려하는 것이 중요합니다.
최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 좋아하고, 어떤 사람은 고양이나 토끼, 새, 물고기 등을 선호하기도 합니다. 반려동물을 선택할 때는 자신의 생활 스타일, 공간, 시간
최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 좋아하고, 어떤 사람은 고양이나 토끼, 새, 물고기 등을 더 좋아할 수 있지요. 반려동물을 선택할 때는 자신의 생활 스타일, 공간
최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 좋아하고, 어떤 사람은 고양이나 토끼, 새, 물고기 등을 더 좋아할 수 있지요. 반려동물을 선택할 때는 자신의 생활 스타일, 공간
최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 좋아하고, 어떤 사람은 고양이나 새, 물고기, 토끼 등 다른 동물을 더 좋아할 수 있지요. 반려동물을 선택할 때는 자신의 생활
최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 좋아하고, 어떤 사람은 고양이나 토끼, 새, 물고기 등을 선호하기도 합니다. 반려동물을 선택할 때는 자신의 생활 스타일, 공간, 시간
최고의 반려동물은 사람마다 다를 수 있습니다. 어떤 사람은 강아지를 좋아하고, 어떤 사람은 고양이를 더 좋아하기도 하죠. 반려동물을 선택할 때는 자신의 생활 스타일, 성격, 환경 등을 

# 매개변수: n
**설명**: 각 프롬프트에 대해 생성할 응답의 수를 지정합니다. \
**기본값**: 1 \
**예제**: n = 3 

---
**참고**: 이 매개변수는 여러 응답을 생성하므로 토큰 할당량을 빠르게 소모할 수 있습니다. 신중하게 사용하고 max_tokens 및 stop 설정이 적절한지 확인하세요.

In [6]:
response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": "최고의 반려동물은"}],
                max_tokens=60,
                n=2
        )

for index, c in enumerate(response.choices):
    print(index, c.message.content)
    print("\n" + "-"*80 + "\n")

0 최고의 반려동물은 사람마다 다를 수 있습니다. 개인의 생활 방식, 성격, 취향에 따라 맞는 반려동물이 다르기 때문인데요.

- 강아지: 충성심이 강하고 활발하며 사람과의 교감이

--------------------------------------------------------------------------------

1 "최고의 반려동물"은 개인의 취향과 생활 환경에 따라 다를 수 있습니다. 예를 들어, 개는 충성심이 강하고 활동적이며 사람과의 교감이 깊어 많은 사람들이 선호합니다. 고양이는 독립적이

--------------------------------------------------------------------------------



# 매개변수: presence_penalty
**설명**: 텍스트에 이미 나타난 토큰을 기반으로 새 토큰에 페널티를 부여하여 모델이 새로운 토큰을 사용하도록 유도합니다. \
**값 범위**: -2.0에서 2.0 \
**기본값**: 0 \
**예제**: presence_penalty=0.5

In [7]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "최고의 반려동물은 "}],
                    max_tokens=60,
                    presence_penalty=presence_penalty, 
                    
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Presence Penalty: 0

최고의 반려동물은 사람마다 다를 수 있습니다. 개, 고양이, 토끼, 새 등 각각의 동물이 가진 성격과 특성이 다르기 때문에, 자신의 생활 스타일과 성향에 맞는 반려동물을 선택하는 것이 중요합니다.

--------------------------------------------------------------------------------

Presence Penalty: 0.5

최고의 반려동물은 사람마다 다를 수 있지만, 보통 개나 고양이가 많이 사랑받습니다. 개는 충성심이 강하고 사람과 교감하기 좋아 많은 사람들에게 좋은 친구가 됩니다. 고양이는 독립적이면서도 애교

--------------------------------------------------------------------------------

Presence Penalty: 1.0

최고의 반려동물은 개인의 라이프스타일, 성격, 환경에 따라 다릅니다. 어떤 사람에게는 강아지가 최고의 반려동물일 수 있고, 어떤 사람에게는 고양이, 토끼, 햄스터, 또는 물고기가

--------------------------------------------------------------------------------

Presence Penalty: 1.5

"최고의 반려동물"은 사람마다 다를 수 있어요. 어떤 분은 강아지가 최고라고 생각하고, 어떤 분은 고양이를 좋아하기도 하죠. 또한 햄스터, 토끼, 새, 물고기 등 다양한 반려동물이

--------------------------------------------------------------------------------

Presence Penalty: 2.0

최고의 반려동물은 사람마다 다를 수 있습니다. 개, 고양이, 토끼, 새, 물고기 등 다양한 동물이 각자의 매력을 가지고 있기 때문입니다. 자신과 생활 스타일, 성격, 취향에 맞는 반려동물을 선택

--------------------------

# 매개변수: frequency_penalty
**설명**: 텍스트에 이미 나타난 빈도를 기반으로 새 토큰에 페널티를 부여하여 동일한 줄을 반복할 가능성을 줄입니다. \
**값 범위**: -2.0에서 2.0 \
**기본값**: 0 \
**예제**: frequency_penalty=0.5

#### 탐색할 사용 사례
1. **응답 비교** \
여러 응답을 생성하여 사용 사례에 가장 적합한 응답을 선택하세요.

2. **다양성 증가** \
다양한 응답을 얻기 위해 여러 응답을 생성하세요. 이는 창의적인 응용 프로그램에 유용합니다.

3. **강건성 향상** \
여러 응답을 생성하여 일관성과 정확성을 보장하세요.

#### 모범 사례
1. **프롬프트 길이 최적화** \
프롬프트를 간결하지만 정보가 충분하도록 유지하여 모델이 충분한 컨텍스트를 가지도록 하세요.

2. **Temperature 및 Top_p 조정** \
결정론적 응답과 창의적 응답 간의 균형을 맞추기 위해 이 매개변수를 사용하세요.

3. **토큰 사용량 모니터링** \
max_tokens 매개변수를 신중히 설정하여 비용과 응답 길이를 관리하세요.

4. **중지 시퀀스 사용** \
모델이 텍스트 생성을 중지해야 할 위치를 정의하여 원하는 컨텍스트 내에서 출력을 제어하세요.

5. **여러 응답 생성** \
n 매개변수를 사용하여 여러 응답을 생성하고 필요에 가장 적합한 응답을 선택하세요.